In [1]:
import os
import pickle
from skimage.io import imread
from skimage.transform import resize
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,mean_absolute_error, classification_report, confusion_matrix, average_precision_score

# Data loading
input_dir = r"C:\Users\Dell\Desktop\ParkEase\clf-data"
categories = ['empty', 'not_empty']
data = []
labels = []

for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(input_dir, category)):
        img_path = os.path.join(input_dir, category, file)
        img = imread(img_path)
        img = resize(img, (15, 15))
        data.append(img.flatten())
        labels.append(category_idx)

data = np.asarray(data)
labels = np.asarray(labels)

# Train/test split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.4 , shuffle=True, stratify=labels)

# Train Decision Tree classifier
classifier = DecisionTreeClassifier(random_state=42)
parameters = [{'max_depth': [3, 5, 7, 10, None], 
               'min_samples_split': [2, 5, 10], 
               'min_samples_leaf': [1, 2, 4],
               'criterion': ['gini', 'entropy']}]
grid_search = GridSearchCV(classifier, parameters, cv=5)
grid_search.fit(x_train, y_train)

# Test performance
best_estimator = grid_search.best_estimator_
print("Best parameters found:", grid_search.best_params_)
y_prediction = best_estimator.predict(x_test)
score = accuracy_score(y_test, y_prediction)
print('{}% of samples were correctly classified'.format(str(score * 100)))

mae = mean_absolute_error(y_test, y_prediction)
absolute_errors = np.abs(y_test - y_prediction)
sae = np.std(absolute_errors)
print("Mean Absolute Error (MAE):", mae)
print("Standard Absolute Error (SAE):", sae)
# Detailed metrics
print("Precision:", precision_score(y_test, y_prediction))
print("Recall:", recall_score(y_test, y_prediction))
print("F1 Score:", f1_score(y_test, y_prediction))
print("Average Precision:", average_precision_score(y_test, y_prediction))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_prediction))
print("\nClassification Report:\n", classification_report(y_test, y_prediction))

Best parameters found: {'criterion': 'entropy', 'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 2}
99.68468468468468% of samples were correctly classified
Mean Absolute Error (MAE): 0.003153153153153153
Standard Absolute Error (SAE): 0.05606434498275988
Precision: 0.9967186218211649
Recall: 0.9975369458128078
F1 Score: 0.9971276159212146
Average Precision: 0.9956150011975872
Confusion Matrix:
 [[ 998    4]
 [   3 1215]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      1002
           1       1.00      1.00      1.00      1218

    accuracy                           1.00      2220
   macro avg       1.00      1.00      1.00      2220
weighted avg       1.00      1.00      1.00      2220

